In [16]:
import os

In [17]:
%pwd


'd:\\Machine Learning\\Wine-Classification'

In [18]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
      root_dir: Path
      test_data: Path
      model_path: Path
      all_params: dict
      metric_file_name: Path
      target_column: str

In [19]:
from src.wineModel.utils import *
from src.wineModel.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
      ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directory([self.config.artifacts_root])
      
      def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config = self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directory([config.root_dir])

            model_eval = ModelEvaluationConfig(
                  root_dir = config.root_dir,
                  test_data = config.test_data,
                  model_path = config.model_path,
                  all_params = params,
                  metric_file_name = config.metric_file_name,
                  target_column = schema.name
            )

            return model_eval

In [20]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error


class ModelEvaluation:
      def __init__(self, config: ModelEvaluationConfig):
            self.config = config

      def eval_mertic(self, actual, pred):
            mae = mean_absolute_error(actual, pred)
            mse = mean_squared_error(actual, pred)
            r2 = r2_score(actual, pred)

            return mae, mse, r2

      def test_model(self):
            test_data = pd.read_csv(self.config.test_data)
            model = joblib.load(self.config.model_path)

            x_val = test_data.drop(columns='quality')
            y_val = test_data[['quality']]

            y_pred = model.predict(x_val)

            (mae, mse, r2) = self.eval_mertic(y_val, y_pred)

            scoring = {
                  "MAE": mae,
                  "MSE": mse,
                  'R2 score': r2
            }

            save_json(path=Path(self.config.metric_file_name), data=scoring)

In [21]:
try:
      config = ConfigurationManager()
      model_evaluation_config = config.get_model_evaluation_config()
      model_eval = ModelEvaluation(config=model_evaluation_config)
      model_eval.test_model()
except Exception as e:
      raise e

[2026-02-12 20:12:36,388: INFO: __init__: yaml file: config\config.yaml is loaded successfully]
[2026-02-12 20:12:36,388: INFO: __init__: yaml file: params.yaml is loaded successfully]
[2026-02-12 20:12:36,388: INFO: __init__: yaml file: schema.yaml is loaded successfully]
[2026-02-12 20:12:36,388: INFO: __init__: Directory is created at: artifacts]
[2026-02-12 20:12:36,388: INFO: __init__: Directory is created at: artifacts/model_evluation]
[2026-02-12 20:12:36,416: INFO: __init__: json file saved at: artifacts\model_evluation\metrics.json]
